In [5]:
import matplotlib.pyplot as plt
import itertools
import random
import copy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
from operator import itemgetter
from itertools import groupby
%matplotlib inline
import statistics

In [113]:
class world():
    def __init__(self, energy = 800*20, time = 0, outside_kill_zone = 500, initial_species_mean = 500, sd=50, loci = 20, total_pop = 2000, replication_ratio = 1.2):
        '''
        NOTE; SEXUAL IS REALLY HERMAPHRODITIC IN THIS PROGRAM
        we want here a fitness landscape, how much energy the world can provide for each time unit - 
        which has some notion of disaster and greatness incorporated into it - i.e. it fluctuates.
        The fitness function defines what proportion of teh energy each organism recieves
        then the energy defines how likely an organism is to survive - maybe also how likely they are to reproduce
        '''
        self.energy = energy # LATER WE MAKE THIS A SORTOFF RANDOM FUNCTION OF TIME PERIOD
        self.loci = loci
        self.outside_kill_zone = outside_kill_zone
        self.initial_species_mean = initial_species_mean
        self.total_pop_mat = (np.random.normal(self.initial_species_mean, sd, (loci, total_pop))).astype(int)
        self.species_sep_index = int(total_pop/2)
        self.fitness_array = []
        self.replication_ratio = replication_ratio
    
    def kill_organisms(self, kill_list):
        # AGAIN WE NEED TO GENERALISE THIS, BUT FOR NOW IT WORKS WITH TWO SPECIES
        self.total_pop_mat = np.delete(self.total_pop_mat, kill_list, axis=1)
        
        old_index = self.species_sep_index
        self.species_sep_index = old_index - len(np.where(kill_list<old_index))

        
    def outside_region_kill(self):
        #here we want to kill any organisms that fall into the kill zone
        #this kill zone is defined as when any gene number is self.insta_kill_zone away from the orignal mean (1000)
        outside_kill_zone = self.outside_kill_zone
        killer_gene_locations = np.where(abs(self.gen_tot_pop_mat()-self.initial_species_mean)>outside_kill_zone)
        kill_organisms = [x[1] for x in killer_gene_locations]
        self.kill_organisms(kill_organisms)
  
        
    def mutation_stage(self, step = 1, mutation_liklihood = 0.02):
        # OKAY, WHETHER OR NOT A MUTATION IS GOOD CRUCIALL DEPENDS ON THE FITNESS LANDSCAPE
        # IT WILL BE MORE GENERAL IF WE INDEED MAKE MUTATIONS RANDOM AND THE FITNESS LANDSCALE SUCH THAT MOST
        # RANDOM MUTATIONS ARE HARMFUL
        shape = (self.total_pop_mat).shape
        half_liklihood = mutation_liklihood/2
        mutation_matrix = np.random.choice([-step, 0, step], size=shape, p=[half_liklihood, 1-mutation_liklihood,half_liklihood])
        self.total_pop_mat = self.total_pop_mat + mutation_matrix
        
    def calc_fitness_array(self):
        self.fitness_array = np.sum(self.total_pop_mat, axis=0)
        
    def survival_prob(self):
        #for now we will do a simple sum for fitness, this should lead to the population evolving to the corners 
        #of the allowed distribution. Later we will try more interesting and random fitness landscapes
        #with different levels of jaggedness
        self.calc_fitness_array()
        total_fitness = np.sum(self.fitness_array)
        prop_fitness = np.divide(self.fitness_array, total_fitness) # AGAIN THIS NEEDS TO BE RETHOUGHT
        return prop_fitness, total_fitness
    
    def survival_round(self):
        number_surviving = int(self.energy/self.loci)
        curr_population_index = range(((self.total_pop_mat).shape)[1])
        prop_fitness = (self.survival_prob())[0]
        survivor_list = np.random.choice(curr_population_index, number_surviving, replace = False, p = prop_fitness)
        self.total_pop_mat = (self.total_pop_mat)[:,survivor_list]
        print(self.species_sep_index)
        old_index = self.species_sep_index
        print(np.where(survivor_list<old_index)
        print(len(np.where(survivor_list<old_index))
        self.species_sep_index = len(np.where(survivor_list<old_index))
        self.fitness_array = self.fitness_array[survivor_list]
        print(self.species_sep_index)
    
            
    
    def asex_repl(self,asex_pop):
        size_of_parent_gen = asex_pop.shape[1]
        next_gen_size = int(size_of_parent_gen*(10/8))
        #in ASEXUAL REPLICATION, everyone living replicates, then the excess energy is used to
        # ramdomly replicate a few more organisms: next_gen_additionals
        next_gen_additional_size = next_gen_size - size_of_parent_gen
        next_gen_additional_chosen_index = random.sample(range(size_of_parent_gen), next_gen_additional_size)
        next_gen_additions = asex_pop[:, next_gen_additional_chosen_index]
        print(asex_pop)
        print(next_gen_additions)
        next_gen = np.concatenate([asex_pop, next_gen_additions], axis=1)
        return next_gen
    
    def mating_pairs(self, index_to_be_paired_and_mated, sex_pop):
        random.shuffle(index_to_be_paired_and_mated)
        offspring = []
        ones_loci = np.ones(shape = (1, self.loci))
        mating_pairs = zip(index_to_be_paired_and_mated[0::2], index_to_be_paired_and_mated[1::2])
        for pair in mating_pairs:
            (a,b) = pair
            parent_1 = sex_pop[:,a]
            parent_2 = sex_pop[:,b]
            parent_1_gene_index = np.random.randint(2, size=(1,self.loci))
            parent_1_genes = np.multiply(parent_1_gene_index, parent_1)
            
            parent_2_gene_index = np.subtract(ones_loci, parent_1_genes)
            parent_2_genes = np.multiply(parent_2_gene_index, parent_2)
            
            offspring_genes = np.add(parent_1_genes, parent_2_genes)
            offspring.append(offspring_genes)
        
        return offspring
    
    def sex_repl(self, sex_pop, mating_rounds = 2):
        #CURRENTLY GROSSLY INEFFICIENT, WE NEED TO MAKE THIS FASTER SOMEHOW
        
        size_of_parent_gen = sex_pop.shape[1]
        next_gen_size = int(size_of_parent_gen*(10/8))
        next_gen_additional_size = next_gen_size - size_of_parent_gen
        #WE RUN TWO BASIC ROUNDS OF REPLICATION
        #WHERE EVERYONE IS RANDOMLY PAIRED TO PRODUCE ONE OFFSPRING
        # WE NEED TO RECONSIDER THIS TO ACCOUNT FOR THE FACT THAT SEXUAL REPLICATION
        #IS A LOT SLOWER THAN ASEXUAL REPLICATION
        parents_index = list(range(size_of_parent_gen))
        
        if size_of_parent_gen%2 == 1:
            random.shuffle(parents_index)
            del parents_index[-1]
        
        #RIGHT NOW WE ONLY HAVE TWO MATING ROUNDS!, AND THEN THE EXTRA ENERGY ON AN EXTRA NOT FULL ROUND  
        #LATER WE MAY NEED TO GENERALISE TO FEWER
        offspring_from_full_rounds = (self.mating_pairs(parents_index, sex_pop)) + (self.mating_pairs(parents_index, sex_pop))
    
        chosen_for_rereplication = parents_index[:(2*next_gen_additional_size)]
        offspring_extra = self.mating_pairs(chosen_for_rereplication, sex_pop)
        
        total_offspring = np.asarray(offspring_from_full_rounds + offspring_extra)
        
        return total_offspring
        

        
    def replication_round(self):
        #HERE IS THE ONLY PLACE WHERE THE TWO SPECIES ARE CONSIDERED SEPERATELY
        #THIS IS WHERE THE species_sep_index becomes relevant
        #CURRENTLY WE DO REPLICATION NOT BASED ON FITNESS:
        #asex_fitness_array = self.fitness_array[0:self.species_sep_index]
        #total_asex_fitness = np.sum(asex_fitness_array)
        
        pop_size = (self.total_pop_mat).shape[1]
        
        asex_pop = self.total_pop_mat[:, 0:self.species_sep_index]
        sex_pop = self.total_pop_mat[:, self.species_sep_index:]

        new_asex_pop = self.asex_repl(asex_pop)
        new_sex_pop = self.sex_repl(sex_pop)
        self.total_pop_mat = np.concatenate([new_asex_pop,new_sex_pop], axis=1)
        self.species_sep_index = (new_asex_pop.shape)[1]
    

SyntaxError: invalid syntax (<ipython-input-113-3fba58faea52>, line 66)

In [114]:
gia = world(energy = 800*5, time=0, outside_kill_zone=500, initial_species_mean=500, sd = 50, loci=5, total_pop=1000, replication_ratio=1.2)

In [115]:
gia.mutation_stage(step = 1, mutation_liklihood=0.02)
print(gia.total_pop_mat)
gia.survival_round()
gia.replication_round()

[[488 526 540 ... 577 606 509]
 [605 555 483 ... 464 449 471]
 [586 495 505 ... 491 597 481]
 [496 596 498 ... 427 599 560]
 [517 479 503 ... 482 584 489]]
500
1
[[508]
 [436]
 [518]
 [548]
 [505]]
[]
as
(5, 1)
[[508]
 [436]
 [518]
 [548]
 [505]]
(997, 1, 5)


ValueError: all the input arrays must have same number of dimensions